# 对CoH模块的测试

* 日期格式转化

In [4]:
from coh.utils import int_to_ordinal
int_to_ordinal(663)

'on the 27th day'

* LLM筛选文本处理

In [3]:
from coh.utils import parse_ids_to_list
parse_ids_to_list("1, 3, 5", ["asd", "sda", "a", "3", "t", "e", "w"])

['sda', '3', 'e']

* vllm LLM模型加载

In [1]:
from utils import vllm_builder

model_name = "mixtral"

llm, task_params = vllm_builder(model_name)

llm, task_params

INFO 05-28 23:38:16 [__init__.py:239] Automatically detected platform cuda.
WARNING 05-28 23:38:18 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 05-28 23:38:25 [config.py:717] This model supports multiple tasks: {'classify', 'reward', 'embed', 'generate', 'score'}. Defaulting to 'generate'.
INFO 05-28 23:38:26 [gptq_marlin.py:147] Detected that the model can run with gptq_marlin, however you specified quantization=gptq explicitly, so forcing gptq. Use quantization=gptq_marlin for faster inference
INFO 05-28 23:38:26 [gptq_bitblas.py:168] Detected that the model can run with gptq_bitblas, however you specified quantization=gptq explicitly, so forcing gptq. Use quantization=gptq_bitblas for faster inference
WARNING 05-28 23:38:26 [config.py:830] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-28 23:38:26 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 05-28 23:38:26 [cuda.py:93

2025-05-28 23:38:32,460 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


WARNING 05-28 23:38:32 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7fd07fc14070>
INFO 05-28 23:38:32 [parallel_state.py:1004] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0
INFO 05-28 23:38:32 [cuda.py:221] Using Flash Attention backend on V1 engine.
INFO 05-28 23:38:32 [topk_topp_sampler.py:44] Currently, FlashInfer top-p & top-k sampling sampler is disabled because FlashInfer>=v0.2.3 is not backward compatible. Falling back to the PyTorch-native implementation of top-p & top-k sampling.
INFO 05-28 23:38:32 [gpu_model_runner.py:1329] Starting to load model /root/autodl-fs/models/TheBloke/Mixtral-8x7B-Instruct-v0___1-GPTQ/...
WARNING 05-28 23:38:32 [utils.py:168] The model class MixtralForCausalLM has not defined `packed_modules_mapping`, this may lead to incorrect mapping of quantized or ignored modules


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [01:27<00:00, 87.53s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [01:27<00:00, 87.53s/it]



INFO 05-28 23:40:02 [loader.py:458] Loading weights took 87.98 seconds
INFO 05-28 23:40:03 [gpu_model_runner.py:1347] Model loading took 22.1664 GiB and 89.980749 seconds
INFO 05-28 23:40:08 [kv_cache_utils.py:634] GPU KV cache size: 38,048 tokens
INFO 05-28 23:40:08 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 1.16x
INFO 05-28 23:40:08 [core.py:159] init engine (profile, create kv cache, warmup model) took 5.51 seconds
INFO 05-28 23:40:08 [core_client.py:439] Core engine process 0 ready.


(<vllm.entrypoints.llm.LLM at 0x7f2420777a30>,
 {'coh_pick': {'max_tokens': 128,
   'top_p': 1.0,
   'temperature': 0.0,
   'stop': ['</s>', '[/INST]']},
  'coh_predict': {'max_tokens': 256,
   'top_p': 1.0,
   'temperature': 0.0,
   'stop': ['</s>', '[/INST]']},
  'fliter': {'max_tokens': 16,
   'top_p': 1.0,
   'temperature': 0.0,
   'stop': ['</s>', '[/INST]']}})

* transformers LLM模型加载测试

In [6]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# model_name = "./models/TheBloke/Mixtral-8x7B-Instruct-v0___1-GPTQ/"

# tokenizer = AutoTokenizer.from_pretrained(
#     model_name, 
#     trust_remote_code=True,
#     )
# tokenizer.pad_token = tokenizer.eos_token
# llm = AutoModelForCausalLM.from_pretrained(
#     model_name, 
#     trust_remote_code=True,
#     device_map="auto",
#     torch_dtype=torch.float16
#     )


* 数据集加载测试

In [2]:
from utils import Data

contents = Data(
    model_name=model_name,
    llm = llm,
    param = task_params['fliter'],
    
    dataset="ICEWS14_forecasting", 
    add_reverse_relation=True
    )

seen dataset proportion: 0.9309428950863213
unseen dataset proportion: 0.06905710491367861


* LLM预测文本处理

In [3]:
from coh.utils import parse_predict_answer

str = """Possible answers:
1. Independent Corrupt Practices Commission
2. Education (Nigeria)
3. Muslim (Nigeria)
4. Member of the Judiciary (Nigeria)
5. Citizen (Nigeria)
6. Boko Haram
7. Government (Nigeria)
"""

print(parse_predict_answer(str, contents.entity2id))

['independent corrupt practices commission', 'education (nigeria)', 'muslim (nigeria)', 'member of the judiciary (nigeria)', 'citizen (nigeria)', 'boko haram']


* CoH模块加载测试

In [5]:
from coh import CoH

coh = CoH(
    num_entities=contents.num_entities,
    num_relations=contents.num_relations,
    entity2id=contents.entity2id,
    id2entity=contents.id2entity,
    id2relation=contents.id2relation,
    s_his_dict=contents.get_adj_his_format_dict(),
    llm=llm,
    # tokenizer=tokenizer,
    params=task_params.get("coh", {}),
    expand_n=5
)

[2025-05-28 23:52:25] INFO model.py:58: testing LLM...
[2025-05-28 23:52:25] ERROR model.py:108: Error testing LLM: 'coh_predict'
[2025-05-28 23:52:25] INFO model.py:60: LLM is working!


* 取10条数据测试

In [9]:
test_data = contents.test_data
test_data = test_data[:10]

def extract_data(data):
    s, p, o, t = data
    return [s, p, t], o

q, a = zip(*[extract_data(item) for item in test_data])

q[0], a[0]

([np.int64(0), np.int64(18), np.int64(7536)], np.int64(384))

* 推理过程

In [10]:
scores = coh(q)

Processed prompts:   0%|          | 0/21084 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/…

* 样例研究

In [ ]:
scores.shape
scores # (batch_size, num_entities)

q_list = coh.format_his_list([s, p, o, t] for [s, p, t], o in zip(q, a))

# q_list

for score, _q in zip(scores, q_list):
    print("--------------------------------------------------")
    print("target: " + _q + '\n')
    for i in range(len(score)):
        if score[i] != 0:
            print(contents.id2entity[i] + ': ' + str(score[i]))
            print()

--------------------------------------------------
target: Cabinet / Council of Ministers / Advisors (United States)	Consult	Sergey Viktorovich Lavrov	on the 314th day

John Kerry: tensor(0.3543)

Abdullah Abdullah: tensor(0.0110)

Ashraf Ghani Ahmadzai: tensor(0.0001)

Sergey Viktorovich Lavrov: tensor(0.2891)

Algirdas Butkevičius: tensor(0.2315)

--------------------------------------------------
target: Benjamin Netanyahu	Consult	Likud	on the 314th day

China: tensor(0.3543)

Barack Obama: tensor(0.4256)

Xi Jinping: tensor(0.2891)

--------------------------------------------------
target: Japan	Engage in diplomatic cooperation	China	on the 314th day

South Korea: tensor(0.4256)

China: tensor(0.3543)

Other Authorities / Officials (United States): tensor(0.2891)

--------------------------------------------------
target: Japan	Arrest, detain, or charge with legal action	Citizen (South Korea)	on the 314th day

--------------------------------------------------
target: Japan	Make a

* 评估测试

In [ ]:
from utils import Evaluator

eval = Evaluator("coh test")
eval.update(score_batch=scores, true_list=a)

eval.print()

--- coh test ---
MRR (%): 30.72
Hits@1 (%): 20.00
Hits@3 (%): 40.00
Hits@10 (%): 40.00
-------------------------
